In [17]:
import os
from pathlib import Path

import xmlrpc.client
import pandas as pd

api_url = os.environ.get('ODOO_URL_API')
# api_db = os.environ.get('ODOO_DB_API')
api_db = os.environ.get('ODOO_DB_PRUEBA_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')

common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

db_file = 'proveedores_oficiales.xlsx'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/compras/{db_file}'))


In [18]:
proveedores = pd.read_excel(db_file_path_str)
prov_oficiales = proveedores.loc[proveedores['oficial'] == 1][['partner_id', 'partner_name']]
prov_locales = proveedores.loc[proveedores['oficial'] == 0][['partner_id', 'partner_name']]

In [51]:
fields_compras = ['name', 'state','partner_id', 'partner_ref', 'date_approve', 'x_fecha_factura', 'user_id', 'create_uid']

ids_compras = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [[("state", "in", ("purchase", "done"))]])
json_compras = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [ids_compras], {'fields': fields_compras})

In [52]:
data_compras = []

for compra in json_compras:
    new = {}
    new['order_id'] = compra['id']
    new['order_name'] = compra['name']
    new['order_state'] = compra['state']
    new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
    new['partner_id'] = compra['partner_id'][0]
    new['partner_name'] = compra['partner_id'][1]
    new['partner_fact_ref'] = compra['partner_ref']
    new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
    new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
    new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

    data_compras.append(new)

compras = pd.DataFrame(data_compras)
compras['order_date'] = pd.to_datetime(compras['order_date'], format='%Y-%m-%d %H:%M:%S')
compras['partner_fact_date'] = pd.to_datetime(compras['partner_fact_date'], format='%Y-%m-%d')

In [55]:
compras

,order_id,order_name,order_state,order_date,partner_id,partner_name,partner_fact_ref,partner_fact_date,capturista,vendedora
0,2715,P02702,purchase,2024-06-03 20:52:37,5384,Truper,115181263-1,2024-05-30,Alexa Yadira Mazariegos Zunun,Mariana Araceli Carvajal Flores
1,2713,P02700,purchase,2024-06-03 20:15:17,5128,Mercado Libre,2000005832834273,NaT,Alexa Yadira Mazariegos Zunun,Mayra Angelica Parada Manjarrez
2,2712,P02699,purchase,2024-06-03 20:12:48,5228,Agencia Arjona De La Paz,JCO 13004,NaT,Alexa Yadira Mazariegos Zunun,Irma Carvajal Flores
3,2711,P02698,purchase,2024-06-03 20:10:53,5227,Anabell Hernandez Espinoza,A 349,NaT,Alexa Yadira Mazariegos Zunun,Irma Carvajal Flores
4,2710,P02697,purchase,2024-06-03 19:07:56,5367,Inoxidables Gran Baja,1374,NaT,Alexa Yadira Mazariegos Zunun,Brenda Luz Acosta Lopez
...,...,...,...,...,...,...,...,...,...,...
2335,11,P00008,purchase,2024-01-09 17:16:41,5322,Martha Veronica Pablo Ambrocio,A 2197,NaT,<NA>,Elsa Ivette Diaz Leyva
2336,10,P00007,purchase,2024-01-03 18:48:16,5283,Chapa Industrias,AA 1036167,NaT,<NA>,Rosario Martinez Zarate
2337,3,P00003,purchase,2024-01-03 18:47:13,5283,Chapa Industrias,AA 1036166A,NaT,<NA>,Rosario Martinez Zarate
2338,2,P00002,purchase,2024-01-03 00:09:24,5326,Madereria El Pino De Los Cabos,SJCFCO 23,NaT,<NA>,Yamilet Blanco Salas


In [61]:
fields_compras_line = ['order_id', 'date_approve', 'partner_id','product_id', 'product_qty', 'price_unit_discounted']

ids_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", ids_compras)]])
json_compras_line = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [ids_compras_line], {'fields': fields_compras_line})

In [62]:
data_compras_line = []

for line in json_compras_line:
    new = {}
    new['line_id'] = line['id']
    new['order_id'] = line['order_id'][0]
    new['order_name'] = line['order_id'][1]
    new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
    new['partner_id'] = line['partner_id'][0]
    new['partner_name'] = line['partner_id'][1]
    new['product_id'] = line['product_id'][0]
    new['product_name'] = line['product_id'][1]
    new['product_qty'] = line['product_qty']
    new['product_price'] = line['price_unit_discounted']
    
    data_compras_line.append(new)

compras_linea = pd.DataFrame(data_compras_line)
compras_linea['order_date'] = pd.to_datetime(compras_linea['order_date'], format='%Y-%m-%d %H:%M:%S')

compras_linea['oficial'] = compras_linea['partner_id'].isin(prov_oficiales['partner_id'])

In [63]:
compras_linea[(compras_linea['order_date'].dt.month < 6) & (compras_linea['oficial'] == False)]

,line_id,order_id,order_name,order_date,partner_id,partner_name,product_id,product_name,product_qty,product_price,oficial
152,15873,2693,P02680 (SJCFCO 14196),2024-05-31 22:01:45,5326,Madereria El Pino De Los Cabos,12827,[13712] Barrote 2 * 4 * 12 Americano *13712*,60.0,172.41,False
153,15874,2693,P02680 (SJCFCO 14196),2024-05-31 22:01:45,5326,Madereria El Pino De Los Cabos,14484,"[78631] TABLA TEKA NACIONAL SEL 1"" (CAMA) *78631*",26.0,1629.32,False
154,15871,2692,P02679 (SJ 39442),2024-05-31 21:30:28,5369,Acyma Del Noroeste,26800,"[5000] Tablaroca 1/2"" 4x8"" *5000*",3.0,196.99,False
155,15872,2692,P02679 (SJ 39442),2024-05-31 21:30:28,5369,Acyma Del Noroeste,28395,"[12672] Durock 1/2"" 4x8 *12672*",3.0,695.20,False
269,15650,2677,P02664 (ORDEN DE COMPRA),2024-05-31 18:34:44,13732,FRANCISCO LOPEZ SEGOVIA,14480,[14931] Penofin Transparente (Clear) (4Lt) Blu...,5.0,1793.10,False
...,...,...,...,...,...,...,...,...,...,...,...
10996,5,2,P00002 (SJCFCO 23),2024-01-03 00:09:24,5326,Madereria El Pino De Los Cabos,13371,[12749] Triplay Pino 1/4 4'*8' 1/Cara *12749*,10.0,267.25,False
10997,1,1,P00001 (SJCFCO 68),2024-01-03 00:05:53,5326,Madereria El Pino De Los Cabos,14120,"[13786] Tablon 2"" X 12"" X 8´ *13786*",3.0,606.90,False
10998,2,1,P00001 (SJCFCO 68),2024-01-03 00:05:53,5326,Madereria El Pino De Los Cabos,14272,[13508] Tablon Americano 2x12x12 *13508*,2.0,910.34,False
10999,3,1,P00001 (SJCFCO 68),2024-01-03 00:05:53,5326,Madereria El Pino De Los Cabos,14147,[3637] Triplay Pino 3/4 4'*8' 1/Cara *3637*,20.0,646.55,False
